In [1]:
%%capture
!pip install transformers[torch]
!pip install rouge_score

In [2]:
!pip install datasets

In [3]:
from datasets import load_dataset, load_metric
from transformers import T5Tokenizer, T5ForConditionalGeneration, Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq,EarlyStoppingCallback
import numpy as np

In [4]:
from datasets import load_dataset
dataset = load_dataset("multi_news")

The repository for multi_news contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/multi_news.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


Generating train split:   0%|          | 0/44972 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5622 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5622 [00:00<?, ? examples/s]

In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['document', 'summary'],
        num_rows: 44972
    })
    validation: Dataset({
        features: ['document', 'summary'],
        num_rows: 5622
    })
    test: Dataset({
        features: ['document', 'summary'],
        num_rows: 5622
    })
})

In [6]:
dataset['train'].features

{'document': Value(dtype='string', id=None),
 'summary': Value(dtype='string', id=None)}

In [7]:
# Taking the subset of the dataset for the finetuning purpose
train_subset = dataset["train"].select(range(10000))
validation_subset = dataset["validation"].select(range(1000))
test_subset = dataset["test"].select(range(1000))

In [8]:
checkpoint = 't5-small'
tokenizer = T5Tokenizer.from_pretrained(checkpoint)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [9]:
def preprocess_function(examples):
    inputs = ["summarize: " + doc for doc in examples["document"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], max_length=150, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [10]:
# Tokenize datasets
tokenized_train = train_subset.map(preprocess_function, batched=True)
tokenized_validation = validation_subset.map(preprocess_function, batched=True)
tokenized_test = test_subset.map(preprocess_function, batched=True)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [11]:
# Load Model
model = T5ForConditionalGeneration.from_pretrained(checkpoint)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [12]:
# Data collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# ROUGE Metrics

ROUGE (Recall-Oriented Understudy for Gisting Evaluation) is a set of metrics used to evaluate the quality of summaries by comparing them to reference summaries (typically human-generated). ROUGE is particularly popular in the field of natural language processing for tasks such as summarization. The metrics focus on different aspects of the generated summary and provide insights into its quality. The main ROUGE metrics include:

## ROUGE-N
Measures the overlap of n-grams between the candidate summary and the reference summary. The most common versions are ROUGE-1 (unigrams) and ROUGE-2 (bigrams).

### ROUGE-1
Counts the overlap of single words.
- **ROUGE-1 Recall**:
  $$
  \text{ROUGE-1 Recall} = \frac{\text{Number of overlapping unigrams}}{\text{Total unigrams in reference summary}}
  $$
- **ROUGE-1 Precision**:
  $$
  \text{ROUGE-1 Precision} = \frac{\text{Number of overlapping unigrams}}{\text{Total unigrams in candidate summary}}
  $$
- **ROUGE-1 F1-Score**:
  $$
  \text{ROUGE-1 F1-Score} = 2 \times \frac{\text{ROUGE-1 Recall} \times \text{ROUGE-1 Precision}}{\text{ROUGE-1 Recall} + \text{ROUGE-1 Precision}}
  $$

**Example Calculation for ROUGE-1:**

Given a reference summary "The cat sat on the mat." and a candidate summary "The cat is on the mat.", calculate ROUGE-1:
- Unigrams in Reference: {The, cat, sat, on, the, mat}
- Unigrams in Candidate: {The, cat, is, on, the, mat}
- Overlap: {The, cat, on, the, mat}
- Recall: $ \frac{5}{6} $
- Precision: $ \frac{5}{6} $
- F1-Score: $ 2 \times \frac{5/6 \times 5/6}{5/6 + 5/6} = 0.833 $

### ROUGE-2
Counts the overlap of two-word sequences.
- **ROUGE-2 Recall**:
  $$
  \text{ROUGE-2 Recall} = \frac{\text{Number of overlapping bigrams}}{\text{Total bigrams in reference summary}}
  $$
- **ROUGE-2 Precision**:
  $$
  \text{ROUGE-2 Precision} = \frac{\text{Number of overlapping bigrams}}{\text{Total bigrams in candidate summary}}
  $$
- **ROUGE-2 F1-Score**:
  $$
  \text{ROUGE-2 F1-Score} = 2 \times \frac{\text{ROUGE-2 Recall} \times \text{ROUGE-2 Precision}}{\text{ROUGE-2 Recall} + \text{ROUGE-2 Precision}}
  $$

**Example Calculation for ROUGE-2:**

Using the same reference and candidate summaries:
- Bigrams in Reference: {The cat, cat sat, sat on, on the, the mat}
- Bigrams in Candidate: {The cat, cat is, is on, on the, the mat}
- Overlap: {The cat, on the, the mat}
- Recall: $ \frac{3}{5} = 0.600 $
- Precision: $ \frac{3}{5} = 0.600 $
- F1-Score: $ 2 \times \frac{0.6 \times 0.6}{0.6 + 0.6} = 0.600 $

## ROUGE-L
Measures the longest common subsequence (LCS) between the candidate and reference summaries. This captures the longest sequence of words that appear in both summaries in the same order, reflecting the importance of sentence-level structure.
- **ROUGE-L Recall**:
  $$
  \text{ROUGE-L Recall} = \frac{\text{LCS}}{\text{Total words in reference summary}}
  $$
- **ROUGE-L Precision**:
  $$
  \text{ROUGE-L Precision} = \frac{\text{LCS}}{\text{Total words in candidate summary}}
  $$
- **ROUGE-L F1-Score**:
  $$
  \text{ROUGE-L F1-Score} = 2 \times \frac{\text{ROUGE-L Recall} \times \text{ROUGE-L Precision}}{\text{ROUGE-L Recall} + \text{ROUGE-L Precision}}
  $$

**Example Calculation for ROUGE-L:**

Using the same reference and candidate summaries:
- LCS: "The cat on the mat"
- Recall: $ \frac{5}{6} \approx 0.833 $
- Precision: $ \frac{5}{6} \approx 0.833 $
- F1-Score: $ 2 \times \frac{0.833 \times 0.833}{0.833 + 0.833} = 0.833 $


In [13]:
# Define compute_metrics function
rouge = load_metric("rouge")

def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    # Decode the predictions and labels
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    # Compute ROUGE scores
    rouge_output = rouge.compute(predictions=pred_str, references=label_str, use_stemmer=True)

    # Aggregate the ROUGE scores
    result = {key: value.mid.fmeasure * 100 for key, value in rouge_output.items()}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in pred_ids]
    result["gen_len"] = np.mean(prediction_lens)

    return result

/tmp/ipykernel_36/2555908495.py:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge = load_metric("rouge")


The repository for rouge contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/rouge.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


In [14]:
# Seq2Seq training arguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./results",             # Directory to save model checkpoints and logs
    evaluation_strategy="epoch",        # Evaluate the model at the end of each epoch
    save_strategy="epoch",              # Save the model at the end of each epoch
    learning_rate=2e-5,                 # Learning rate for the optimizer
    per_device_train_batch_size=16,     # Batch size for training
    per_device_eval_batch_size=16,      # Batch size for evaluation
    weight_decay=0.01,                  # Weight decay for regularization
    save_total_limit=3,                 # Limit the total number of checkpoints saved
    num_train_epochs=10,                 # Number of training epochs
    predict_with_generate=True,         # Use generation mode for prediction
    generation_max_length=150,          # Maximum length for generated sequences
    generation_num_beams=4,             # Number of beams for beam search during generation
    load_best_model_at_end=True,        # Load the best model when early stopping is triggered
    metric_for_best_model="eval_loss",  # Metric to monitor for early stopping (can be adjusted)
    greater_is_better=False             # Whether a higher metric value is better (for loss, lower is better)
)
# Add early stopping callback
early_stopping_callback = EarlyStoppingCallback(early_stopping_patience=3)

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [15]:
## Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model=model,                       # The model to be trained
    args=training_args,                # Training arguments defined with Seq2SeqTrainingArguments
    train_dataset=tokenized_train,     # The training dataset
    eval_dataset=tokenized_validation, # The evaluation dataset
    data_collator=data_collator,       # The data collator for processing data batches
    tokenizer=tokenizer,               # The tokenizer used for preprocessing
    compute_metrics=compute_metrics,   # The function to compute evaluation metrics
)

In [16]:
# Train the model
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,3.088812,33.825678,10.091313,19.385910,19.396628,131.264000
2,3.487000,3.021594,36.014065,11.169121,20.460082,20.453791,138.120000
3,3.487000,2.990618,36.247008,11.357827,20.663479,20.669243,138.632000
4,3.235400,2.972706,36.725245,11.542169,20.949182,20.945765,139.433000
5,3.186300,2.958589,36.697025,11.653253,20.928116,20.923615,140.189000
6,3.186300,2.951110,36.858440,11.742683,21.139481,21.137748,140.747000
7,3.162400,2.944103,36.949013,11.836197,21.238838,21.250789,140.994000
8,3.146200,2.940566,37.085510,11.838781,21.244715,21.258299,141.331000
9,3.146200,2.938292,37.075695,11.858777,21.230590,21.247206,140.901000
10,3.140900,2.937616,37.144967,11.925938,21.301345,21.314734,141.081000


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0

TrainOutput(global_step=3130, training_loss=3.222881865577576, metrics={'train_runtime': 7182.5892, 'train_samples_per_second': 13.923, 'train_steps_per_second': 0.436, 'total_flos': 1.35341801472e+16, 'train_loss': 3.222881865577576, 'epoch': 10.0})

In [17]:
# Evaluate the model on validation set
trainer.evaluate()

# Evaluate the model on test set
test_results = trainer.evaluate(eval_dataset=tokenized_test)

print(test_results)

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'eval_loss': 2.902773380279541, 'eval_rouge1': 37.35992631839289, 'eval_rouge2': 12.182034188128751, 'eval_rougeL': 21.406795646177123, 'eval_rougeLsum': 21.38270299446754, 'eval_gen_len': 141.366, 'eval_runtime': 379.1725, 'eval_samples_per_second': 2.637, 'eval_steps_per_second': 0.084, 'epoch': 10.0}


In [18]:
import torch
# Select a specific data point from the test dataset
test_index = 0  # Change this index to the specific data point you want to summarize
example_text = dataset["test"][test_index]["document"]

# Preprocess the input text
input_text = "summarize: " + example_text
inputs = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
inputs = inputs.to(device)
# Generate the summary
summary_ids = model.generate(inputs, max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)

# Decode the generated summary
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print("Original Text:\n", example_text)
print("\nGenerated Summary:\n", summary)

Original Text:
 GOP Eyes Gains As Voters In 11 States Pick Governors 
 
 Enlarge this image toggle caption Jim Cole/AP Jim Cole/AP 
 
 Voters in 11 states will pick their governors tonight, and Republicans appear on track to increase their numbers by at least one, with the potential to extend their hold to more than two-thirds of the nation's top state offices. 
 
 Eight of the gubernatorial seats up for grabs are now held by Democrats; three are in Republican hands. Republicans currently hold 29 governorships, Democrats have 20, and Rhode Island's Gov. Lincoln Chafee is an Independent. 
 
 Polls and race analysts suggest that only three of tonight's contests are considered competitive, all in states where incumbent Democratic governors aren't running again: Montana, New Hampshire and Washington. 
 
 While those state races remain too close to call, Republicans are expected to wrest the North Carolina governorship from Democratic control, and to easily win GOP-held seats in Utah, North